# Motion Sensor Controller

This code controls the motion sensors and sends data to the web appplication.

spacing specifies the distance between the sensors, so that the velocity may be calculated

In [ ]:
from pynq.overlays.base import BaseOverlay
import time
base = BaseOverlay('base.bit')
spacing = 1.8

This cell loads config data from this directory, most notibly the server address.

In [ ]:
import json
config = json.load(open('config.json'))

This is the function responsible for sending data to the server. It runs concurrently in a separate thread, to allow the sensor code to run in parallel.

In [87]:
from concurrent.futures import ThreadPoolExecutor
from requests_futures.sessions import FuturesSession
executor = ThreadPoolExecutor(max_workers=4)
server = config['server']
session = FuturesSession(executor=executor)
f_params = session.post(server + '/register?lot=1')
def log_to_server(n):
    params = f_params.result().json()
    if n > 0:
        params["enter"] = n
    else:
        params["exit"] = -n
    session.post(server + "/report", json=params)

Set up the sensors and debugging outputs.

In [88]:
from pynq.lib.pmod import *
from pynq.lib.arduino import Grove_Buzzer as A_Grove_Buzzer, ARDUINO_GROVE_G1

In [89]:
sensors = [Grove_PIR(base.PMODA, port) for port in (PMOD_GROVE_G1, PMOD_GROVE_G2)]
bar = Grove_LEDbar(base.PMODB, PMOD_GROVE_G4)
buzz = A_Grove_Buzzer(base.ARDUINO, ARDUINO_GROVE_G1)

functions to play ascending and descending dubugging tones

In [90]:
def tone_asc():
    buzz.play_tone(494, 428)
    buzz.play_tone(659, 428)
    
def tone_desc():
    buzz.play_tone(659, 428)
    buzz.play_tone(494, 428)

the actual code -- pretty simple: if something triggered the other sensor in the past 10 seconds, then trigger the direction of the motion

In [100]:
ct = 0
detect = [0 for s in sensors]
last = [s.read() for s in sensors]
while True:
    display = 0
    for m, i, s in zip([0b1111100000, 0b0000011111], [0, 1], sensors):
        r = s.read()
        clock = time.process_time()
        display |= r and m
        if r != last[i]:
            # change in sensor output
            last[i] = r
            if r and detect[not i]:
                ct += [-1, 1][i]
                executor.submit([tone_desc, tone_asc][i])
                log_to_server([-1, 1][i])
                print("in" if i else "out", "| count", ct, "|", spacing/(clock - detect[not i]), "m/s")
                detect[not i] = 0
            elif r:
                detect[i] = clock
    detect = [0 if clock - 10 > v else v for v in detect]
    bar.write_binary(display)

in | count 1 | 1.0361491162586316 m/s
out | count 0 | 1.3318424294547664 m/s
in | count 1 | 0.8658359935872015 m/s
in | count 2 | 0.3832279922578756 m/s
out | count 1 | 0.21347056263470673 m/s
out | count 0 | 1.1865558046031612 m/s
in | count 1 | 1.3201443355202553 m/s
in | count 2 | 0.4689964628926911 m/s
in | count 3 | 1.0293425508413887 m/s
in | count 4 | 1.488308677993269 m/s
out | count 3 | 1.5143293388505308 m/s
in | count 4 | 1.6877125041055354 m/s
out | count 3 | 1.5454213448015799 m/s
in | count 4 | 1.4744140711902434 m/s
out | count 3 | 1.6704366767525378 m/s
out | count 2 | 0.5034753093692744 m/s
out | count 1 | 0.7687778782413818 m/s
out | count 0 | 0.9922305295670412 m/s
out | count -1 | 0.43994060464540824 m/s
out | count -2 | 1.3091595004821353 m/s
out | count -3 | 1.3766780793723683 m/s
out | count -4 | 1.5020924485772602 m/s
out | count -5 | 1.4744049734848548 m/s


KeyboardInterrupt: 